# Абстрактные классы и полиморфизм

**Задача**. Нужно разработать механизм, который позволит выводить на консоль геометрические фигуры, который можно легко дополнять новыми описаниями. Самая простая реализация:

In [1]:
def draw(figures):
    for fig in figures:
        fig.draw()

Данная функция ожидает список объектов, у которых должен быть метод `draw` без аргументов. Этот подход является вполне pythonic, но для того, чтоб понять, что именно принимает функция (или чтоб научить IDE подсказывать ошибки), неплохо было бы завести базовый тип, чтоб можно было использовать type hinting:

In [11]:
from typing import List

class Figure:
    def draw(self):
        pass

def draw(figures: List[Figure]):
    for fig in figures:
        fig.draw()

Но используя такой подход, нам никто не помешает создать объект класса `Figure` (хотя с логической точки зрения в этом нет смысла).

In [3]:
figures = [Figure(), Figure(), Figure()]
draw(figures)

**Абстрактный класс** — базовый класс, который не предполагает создания экземпляров. Абстрактный класс может содержать абстрактные методы и свойства. Абстрактный метод не реализуется для класса, в котором описан, однако должен быть реализован для его неабстрактных потомков. Абстрактные классы представляют собой наиболее общие абстракции, то есть имеющие наибольший объём и наименьшее содержание.

В динамических языках (таких как Python), абстрактные классы представляют собой своеобразный способ документирования кода и помогают ограничить (decouple) взаимодействие отдельных абстракций в программе (классов).

Абстрактные классы реализуют на практике один из принципов ООП — **полиморфизм**. Простыми словами, **полиморфизм** – это способность функции обрабатывать данные разных типов.

В Python поддержка абстрактных классов реализуется на уровне стандартной библиотеки с помощью модуля `abc` (начиная с Python 2.6). Абстрактные методы обозначаются декоратором `@abstractmethod`, а абстрактный класс должен быть создан с использованием *метакласса* `ABCMeta`.

In [5]:
from typing import List
from abc import ABCMeta, abstractmethod

class Figure(metaclass=ABCMeta):
    
    @abstractmethod
    def draw(self):
        pass

def draw(figures: List[Figure]):
    for fig in figures:
        fig.draw()

Теперь создать объект класса `Figure` мы не сможем:

In [6]:
figures = [Figure(), Figure(), Figure()]
draw(figures)

TypeError: Can't instantiate abstract class Figure with abstract methods draw

Кроме того, если мы забудем переопределить один из абстрактных методов в классах-наследниках, тоже получим ошибку (т.к. если в классе есть хотя бы один абстрактный метод, этот класс является абстрактным). Это позволяет строить иерархию абстрактных классов, каждый из которых реализует какую-то часть логики.

In [15]:
from typing import List
from abc import ABCMeta, abstractmethod

class Figure(metaclass=ABCMeta):

    @abstractmethod
    def draw(self):
        pass

class Square(Figure):
    def __init__(self, size):
        self.size = 3
        
    def draw(self):
        print(("*" * self.size + "\n") * self.size)

###

def draw(figures: List[Figure]):
    for fig in figures:
        fig.draw()

figures = [Square(3)]
draw(figures)

***
***
***



Кроме *абстрактных методов*, также можно задавать *абстрактные свойства*.

In [16]:
from abc import ABCMeta, abstractmethod, abstractproperty

class Figure(metaclass=ABCMeta):

    @abstractproperty
    def square(self):
        """ Площадь фигуры. """
    
    @abstractmethod
    def draw(self):
        """ Отрисовка фигуры. """


**Абстрактный класс**, который содержит только абстрактные методы, называется **интерфейсом**.

Данных подход нужно использовать в том случае, если вы хотите запретить создавать объекты абстрактного класса. Кроме этого, можно отметить только некоторые методы как нереализованные (чтоб получить ошибку при попытке их вызова).

In [22]:
from typing import List

class Figure:
    def draw(self):
        raise NotImplementedError()

def draw(figures: List[Figure]):
    for fig in figures:
        fig.draw()

figures = [Figure(), Figure(), Figure()]
draw(figures)
# Figure().draw()

NotImplementedError: 

# Метаклассы

**Метаклассы** – это черная магия, о которой 99% пользователей вообще не нужно задумываться. Если вы думаете о том, нужно ли вам их использовать – они вам не нужны (а те, кому они действительно нужны, точно знают, зачем они им, и не нуждаются в объяснениях, почему).

Кто недостаточно испугался – прочитайте https://habrahabr.ru/post/145835/ и забудьте.

В Python метакласс – это штука, которая создает классы. И эта штука позволяет:
* делать дополнительные проверки при создании классов как `ABCMeta` проверяет, что все абстрактные методы и свойства реализованы;
* добавлять новые методы и свойства в класс на лету, или менять их (т.е. делать так, что в коде будет написано одно, а работать оно будет по-другому);
* делать еще много чего, адекватность которого зависит от того, какими веществами увлекается программист, ними пользующийся.

In [23]:
class Person:
    name = "John Doe"

p = Person()
print(type(Person))
print(p.name)

<class 'type'>
John Doe


In [24]:
Person = type('Person', (), {"name": "John Doe"})

p = Person()
print(type(Person))
print(p.name)

<class 'type'>
John Doe


Все метаклассы должны наследоваться от класса `type`.

# Синглтоны (Singleton)

**Синглтон** – порождающий шаблон проектирования, гарантирующий, что в однопоточном приложении будет только один экземпляр некоторого класса, и предоставляющий глобальную точку доступа к этому экземпляру.

Есть мнение, что синглтон это антипаттерн, которые нужно стараться не использовать, потому что он может принести больше проблем, чем пользы (в частности в многопоточных приложениях).

## Способы реализации синглтона в Python

### Способ №1 – плохой

In [31]:
class Singleton(type):
    """ Метакласс для создание синглтонов. """
    __instance = None

    def __call__(cls, *args, **kw):
        if cls.__instance is None:
            cls.__instance = super(Singleton, cls).__call__(*args, **kw)
        return cls.__instance

class SomeService(metaclass=Singleton):
    x = 1

    def say_hello(self):
        print("Hello!")
        
obj1 = SomeService()
obj2 = SomeService()

obj1.x = 2
print(obj2.x)

print(obj1 is obj2)
print(id(obj1))
print(id(obj2))

2
True
4539703704
4539703704


### Способ №2 – немного лучше

In [35]:
import functools

def singleton(cls):
    """ Декоратор, для преобразования класса в Singleton """
    instances = {}
    
    @functools.wraps(cls)
    def decorator():
        if cls not in instances:
            instances[cls] = cls()
        return instances[cls]

    return decorator

@singleton
class SomeService:
    def say_hello(self):
        print("Hello!")

obj1 = SomeService()
obj2 = SomeService()

print(obj1 is obj2)
print(id(obj1))
print(id(obj2))

True
4539704712
4539704712


### Способ №3 – почти идеальный

Создать фабричный метод для получения объекта. Обратите внимание, что мы явно не запрещаем создать несколько объектов класса.

In [38]:
class SomeService:

    @classmethod
    def instance(cls):
        if not hasattr(cls, "__instance"):
            setattr(cls, "__instance", cls())
        return getattr(cls, "__instance")
    
    def say_hello(self):
        print("Hello!")

obj1 = SomeService.instance()
obj2 = SomeService.instance()

print(obj1 is obj2)
print(id(obj1))
print(id(obj2))        
        
SomeService.instance().say_hello()

True
4539737480
4539737480
Hello!


### Способ №4 - идеальный

Если вам нужен только один объект какого-то класса – создайте только один объект этого класса, и используйте его :)

In [39]:
class SomeService:
    def say_hello(self):
        print("Hello!")

obj = SomeService()
obj.say_hello()
obj.say_hello()
obj.say_hello()

Hello!
Hello!
Hello!


# Принципы проектирования ПО

## DRY – Don’t Repeat Yourself

*Каждая часть знания должна иметь единственное, непротиворечивое и авторитетное представление в рамках системы.*

**DRY** ("не повторяйся") — это принцип разработки программного обеспечения, нацеленный на снижение повторения информации различного рода, особенно в системах со множеством слоев абстракции.

Нарушения принципа **DRY** называют **WET — Write Everything Twice** ("пиши все по два раза").

## KISS – Keep It Simple, Stupid

**KISS** в программировании — принцип, запрещающий использование более сложных средств, чем необходимо. Изречение часто вызываемое при обсуждении вопросов проектирования с целью парирования нарастающей функциональности и управления сложностью разработки.

Принцип **KISS** утверждает, что большинство систем работают лучше всего, если они остаются простыми, а не усложняются. Поэтому в области проектирования простота должна быть одной из ключевых целей и следует избегать ненужной сложности.

## YAGNI – You Ain't Gonna Need It

Согласно адептам принципа YAGNI, желание писать код, который не нужен прямо сейчас, но может понадобиться в будущем, приводит к следующим нежелательным последствиям:

* Тратится время, которое было бы затрачено на добавление, тестирование и улучшение необходимой функциональности.
* Новые функции должны быть отлажены, документированы и сопровождаться.
* Новая функциональность ограничивает то, что может быть сделано в будущем — ненужные новые функции могут впоследствии помешать добавить новые нужные.
* Пока новые функции действительно не нужны, трудно полностью предугадать, что они должны делать, и протестировать их. Если новые функции тщательно не протестированы, они могут неправильно работать, когда впоследствии понадобятся.
* Это приводит к тому, что программное обеспечение становится более сложным (подчас чрезмерно сложным).
* Если вся функциональность не документирована, она может так и остаться неизвестной пользователям, но может создать для безопасности пользовательской системы различные риски.
* Добавление новой функциональности может привести к желанию ещё более новой функциональности, приводя к эффекту "снежного кома".

## SOLID

#### The Single Responsibility Principle (Принцип единственной ответственности)
Каждый объект должен иметь одну ответственность и эта ответственность должна быть полностью инкапсулирована в класс. Все его поведения должны быть направлены исключительно на обеспечение этой ответственности.

#### The Open Closed Principle (Принцип открытости/закрытости)
Программные сущности (классы, модули, функции и т. п.) должны быть открыты для расширения, но закрыты для изменения.

#### The Liskov Substitution Principle (Принцип подстановки Барбары Лисков)
Является специфичным определением подтипа, предложенным Барбарой Лисков в 1987 году на конференции в основном докладе под названием "Абстракция данных и иерархия". Идея Лисков о *подтипе* даёт определение понятия замещения — если `S` является подтипом `T`, тогда объекты типа `T` в программе могут быть замещены объектами типа `S` без каких-либо изменений свойств этой программы.

#### The Interface Segregation Principle (Принцип разделения интерфейса)
Клиенты не должны зависеть от методов, которые они не используют. Слишком "толстые" интерфейсы необходимо разделять на более маленькие и специфические, чтобы клиенты маленьких интерфейсов знали только о методах, которые необходимы им в работе. В итоге, при изменении метода интерфейса не должны меняться клиенты, которые этот метод не используют.

#### The Dependency Inversion Principle (Принцип инверсии зависимостей)
* Модули верхних уровней не должны зависеть от модулей нижних уровней. Оба типа модулей должны зависеть от абстракций.
* Абстракции не должны зависеть от деталей. Детали должны зависеть от абстракций.